<a href="https://colab.research.google.com/github/Noam-Coh3n/ModCrypto/blob/main/ProgrammingAssignmentWeek6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Oracle

In [3]:
sign_sock = None
vrfy_sock = None

MAX_PACKET_LEN = 8192
NOT_BINARY_STR_ERR = -1
MISSING_DELIMITER_ERR = -2
ORIGINAL_MSG_ERR = -3

def Oracle_Connect():
    import socket
    global sign_sock
    global vrfy_sock
    sign_sock, vrfy_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM), socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sign_sock.connect(('128.8.130.16', 49104))
        vrfy_sock.connect(('128.8.130.16', 49105))
    except socket.error as e:
        print(e)
        return -1

    print("Connected to server successfully.")

    return 0

def Oracle_Disconnect():
    if not sign_sock or not vrfy_sock:
        print("[WARNING]: You haven't connected to the server yet.")
        return -1

    sign_sock.close()
    vrfy_sock.close()

    print("Connection closed successfully.")

    return 0

# Packet Structure: < message >
# Message may be either a long integer, or a binary string
def Sign(msg):
    if not sign_sock or not vrfy_sock:
        print("[WARNING]: You haven't connected to the server yet.")
        return -1
    if msg < 0:
        print("[ERROR]: Message cannot be negative!")

    if type(msg) is float or type(msg) is int:
        msg = bin(msg)[2:]

    pkt = bytes(msg + "X", "utf-8")
    sign_sock.send(pkt)

    resp = sign_sock.recv(MAX_PACKET_LEN)
    try:
        sigma = int(resp, 2)
    except ValueError as e:
        sigma = int(resp)

    if sigma == NOT_BINARY_STR_ERR:
        print("[ERROR]: Message was not a valid binary string.")
    if sigma == ORIGINAL_MSG_ERR:
        print("[ERROR]: You cannot request a signature on the original message!")

    return sigma

# Packet Structure: < message | ":" | signature >
# Message and signature may be either long integers, or binary strings
def Verify(msg, sigma):
    if not sign_sock or not vrfy_sock:
        print("[WARNING]: You haven't conected to the server yet.")
        return -1
    if msg < 0 or sigma < 0:
        print("[ERROR]: Message and signature cannot be negative!")
        return -1

    if type(msg) is float or type(msg) is int:
        msg = bin(msg)[2:]
    if type(sigma) is float or type(sigma) is int:
        sigma = bin(sigma)[2:]

    pkt = bytes(msg + ":" + sigma + "X", 'utf-8')
    vrfy_sock.send(pkt)

    match = int(vrfy_sock.recv(MAX_PACKET_LEN))

    if match == NOT_BINARY_STR_ERR:
        print("[ERROR]: Message and/or signature were not valid binary strings.")
    elif match == MISSING_DELIMITER_ERR:
        print("[ERROR]: Missing delimiter between message and signature.")
    return match


# Helper

In [8]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None  # modular inverse does not exist
    else:
        return x % m

def ascii_to_int(m):
    val = ""
    for x in m:
        val += hex(ord(x))[2:]
    return int("0x" + val,16)

def ascii_to_bin(m):
    val = ""
    for x in m:
        val += bin(ord(x))[2:].zfill(8)
    return val

# Sample

In [5]:
n = 119077393994976313358209514872004186781083638474007212865571534799455802984783764695504518716476645854434703350542987348935664430222174597252144205891641172082602942313168180100366024600206994820541840725743590501646516068078269875871068596540116450747659687492528762004294694507524718065820838211568885027869

e = 65537

Oracle_Connect()

msg = "Crypto is hard --- even schemes that look complex can be broken"

m = ascii_to_int(msg)

# Should fail, because you're not allowed to query on the original message
sigma = Sign(m)
assert(sigma < 0)

# All other arbitrary messages <= 504 bits should be accepted by the oracle
msg = "Hello, World!"

m = ascii_to_int(msg)

sigma = Sign(m)
if sigma < 0:
    raise SystemExit

if Verify(m,sigma):
    print("Oracle is working properly!")

Oracle_Disconnect()


Connected to server successfully.
[ERROR]: You cannot request a signature on the original message!
Oracle is working properly!
Connection closed successfully.


0

#Solution

In [22]:
from sympy import pollard_rho

n = 119077393994976313358209514872004186781083638474007212865571534799455802984783764695504518716476645854434703350542987348935664430222174597252144205891641172082602942313168180100366024600206994820541840725743590501646516068078269875871068596540116450747659687492528762004294694507524718065820838211568885027869
e = 65537

Oracle_Connect()

msg = "Crypto is hard --- even schemes that look complex can be broken"
m = ascii_to_int(msg)

## your attack here!
p = pollard_rho(m)
x = modinv(Sign(1), n)

sigma = Sign(p) * Sign(m//p) * x % n


ver = Verify(m,sigma)

if ver:
    print("Mission accomplished!")

Oracle_Disconnect()


Connected to server successfully.
Mission accomplished!
Connection closed successfully.


0